In [ ]:
import pandas as pd
import os
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# Ensure working directory is the notebook's location
NOTEBOOK_DIR = os.path.dirname(os.path.abspath("lab.ipynb"))
os.chdir(NOTEBOOK_DIR)
print(f"Working directory: {os.getcwd()}")

In [ ]:
SCRIPT_DIR = (
    os.path.dirname(os.path.abspath(__file__)) if "__file__" in globals() else os.getcwd()
)
PROJECT_ROOT = os.path.dirname(SCRIPT_DIR)
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
CHARTS_DIR = os.path.join(SCRIPT_DIR, "charts")

METRICS = [
    ("pod_10", "response_time", "Response Time (ms)", 10),
    ("pod_10", "replica", "Replica Count", 10),
    ("pod_10", "cpu", "CPU Usage", 10),
    ("pod_10", "memory", "Memory Usage", 10),
    ("pod_20", "response_time", "Response Time (ms)", 20),
    ("pod_20", "replica", "Replica Count", 20),
    ("pod_20", "cpu", "CPU Usage", 20),
    ("pod_20", "memory", "Memory Usage", 20),
]

DEPLOYMENTS = {"HPA": "hpa-flask-app", "RL": "test-flask-app"}

In [ ]:
def get_run_means(base_path):
    """Return dict of runs -> {'hpa': mean, 'rl': mean} for CSVs 1..20 in base_path."""
    runs = {}
    for i in range(1, 21):
        path = os.path.join(base_path, f"{i}.csv")
        if not os.path.exists(path):
            continue
        try:
            # try annotated influx CSV first, then plain CSV
            try:
                df = pd.read_csv(path, skiprows=3, comment="#")
            except Exception:
                df = pd.read_csv(path)
            # drop unnamed cols
            df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
            # detect value column
            if "_value" in df.columns:
                valcol = "_value"
            elif "value" in df.columns:
                valcol = "value"
            else:
                print(f"No value column in {path}, skipping")
                continue
            if "deployment" not in df.columns:
                print(f"No deployment column in {path}, skipping")
                continue
            means = df.groupby("deployment")[valcol].mean()
            runs[i] = {
                "hpa": means.get(DEPLOYMENTS["HPA"], np.nan),
                "rl": means.get(DEPLOYMENTS["RL"], np.nan),
            }
            print(f"Loaded {path}: {len(df)} rows (hpa={runs[i]['hpa']}, rl={runs[i]['rl']})")
        except Exception as e:
            print(f"Skipping {path}: {e}")
    return runs

def build_paired_from_runs(runs_dict):
    """From runs dict return two lists (hpa, rl) only for runs that have both values."""
    hpa, rl = [], []
    for run in sorted(runs_dict.keys()):
        v = runs_dict[run]
        if v is None:
            continue
        h = v.get("hpa", np.nan)
        r = v.get("rl", np.nan)
        if not (pd.isna(h) or pd.isna(r)):
            hpa.append(float(h))
            rl.append(float(r))
    return hpa, rl

def analyze_stats(hpa, rl):
    n = min(len(hpa), len(rl))
    if n < 3: return None  # Need at least 3 samples for Shapiro-Wilk/tests

    x, y = hpa[:n], rl[:n]

    # Normality check (Shapiro-Wilk)
    # H0: Distribution is Normal. If p > 0.05, we fail to reject H0 (assume normal).
    shapiro_hpa = stats.shapiro(x)
    shapiro_rl = stats.shapiro(y)

    is_normal = (shapiro_hpa.pvalue > 0.05) and (shapiro_rl.pvalue > 0.05)

    # Statistical Test
    diff = np.array(x) - np.array(y)

    # Guard: if all differences are zero, no test is meaningful
    if np.allclose(diff, 0):
        return {
            "N": n,
            "HPA Mean": np.mean(x),
            "RL Mean": np.mean(y),
            "Diff (%)": 0.0,
            "Normality P(HPA)": shapiro_hpa.pvalue,
            "Normality P(RL)": shapiro_rl.pvalue,
            "P-Value": 1.0,
            "Significant": False,
            "Test": "N/A (identical values)",
            "Effect Size": 0.0
        }

    if is_normal:
        stat, p_val = stats.ttest_rel(x, y)
        test_name = "Paired t-test"
    else:
        try:
            stat, p_val = stats.wilcoxon(x, y)
            test_name = "Wilcoxon"
        except ValueError as e:
            # Wilcoxon can fail on edge cases (e.g., too few non-zero differences)
            print(f"  Wilcoxon failed ({e}), falling back to Paired t-test")
            stat, p_val = stats.ttest_rel(x, y)
            test_name = "Paired t-test (fallback)"

    # Effect Size (Cohen's d for paired samples)
    std_diff = np.std(diff, ddof=1)
    cohens_d = np.mean(diff) / std_diff if std_diff != 0 else 0

    return {
        "N": n,
        "HPA Mean": np.mean(x),
        "RL Mean": np.mean(y),
        "Diff (%)": (np.mean(y) - np.mean(x)) / np.mean(x) * 100 if np.mean(x) != 0 else np.nan,
        "Normality P(HPA)": shapiro_hpa.pvalue,
        "Normality P(RL)": shapiro_rl.pvalue,
        "P-Value": p_val,
        "Significant": p_val < 0.05,
        "Test": test_name,
        "Effect Size": cohens_d
    }

In [ ]:
def save_normality_plots(hpa, rl, folder, metric_dir, label, pod):
    # Create the directory structure: e.g., result/inferensial_test/pod_10/response_time/
    output_dir = f"{folder}/{metric_dir}"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Saving plots to: {output_dir}/")

    # --- 1. Save INDIVIDUAL plots ---
    # HPA Histogram
    plt.figure(figsize=(6, 4))
    plt.hist(hpa, bins='auto', alpha=0.7, color='blue', edgecolor='black', rwidth=0.85)
    plt.title(f"HPA Histogram - {label}")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/inferensial_histogram_{metric_dir}_hpa_{pod}_pod.png")
    plt.close()

    # HPA Q-Q Plot
    plt.figure(figsize=(6, 4))
    stats.probplot(hpa, dist="norm", plot=plt)
    plt.title(f"HPA Q-Q Plot - {label}")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/inferensial_qq_plot_{metric_dir}_hpa_{pod}_pod.png")
    plt.close()

    # RL Histogram
    plt.figure(figsize=(6, 4))
    plt.hist(rl, bins='auto', alpha=0.7, color='green', edgecolor='black', rwidth=0.85)
    plt.title(f"RL Histogram - {label}")
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/inferensial_histogram_{metric_dir}_rl_{pod}_pod.png")
    plt.close()

    # RL Q-Q Plot
    plt.figure(figsize=(6, 4))
    stats.probplot(rl, dist="norm", plot=plt)
    plt.title(f"RL Q-Q Plot - {label}")
    plt.tight_layout()
    plt.savefig(f"{output_dir}/inferensial_qq_plot_{metric_dir}_rl_{pod}_pod.png")
    plt.close()

    # --- 2. Show SUMMARY plot (Combined) ---
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f"Normality Analysis: {label} ({folder})", fontsize=16)

    # Plot on subplots
    axes[0, 0].hist(hpa, bins='auto', alpha=0.7, color='blue', edgecolor='black', rwidth=0.85)
    axes[0, 0].set_title(f"HPA Histogram")
    axes[0, 0].set_ylabel("Frequency")

    stats.probplot(hpa, dist="norm", plot=axes[0, 1])
    axes[0, 1].set_title(f"HPA Q-Q Plot")

    axes[1, 0].hist(rl, bins='auto', alpha=0.7, color='green', edgecolor='black', rwidth=0.85)
    axes[1, 0].set_title(f"RL Histogram")
    axes[1, 0].set_ylabel("Frequency")

    stats.probplot(rl, dist="norm", plot=axes[1, 1])
    axes[1, 1].set_title(f"RL Q-Q Plot")

    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Adjust for main title
    plt.show()



In [ ]:
def format_scientific(val):
    """Return '-' for missing/NaN, otherwise nicely formatted string for LaTeX."""
    if val is None or pd.isna(val):
        return "-"
    try:
        valf = float(val)
    except Exception:
        return "-"
    if abs(valf) < 0.001 and valf != 0:
        parts = f"{valf:.3e}".split("e")
        mantissa = parts[0]
        exponent = int(parts[1])
        return f"${mantissa} \\times 10^{{{exponent}}}$"
    else:
        return f"{valf:.3f}"

def format_effect_size(val, test_name):
    """Format effect size with appropriate label"""
    if "Wilcoxon" in test_name:
        return f"{abs(val):.3f}"
    else:
        return f"{val:.3f}"

def generate_latex_table(results_df, metric_name, table_label, caption):
    """Generate LaTeX table for a specific metric"""

    # Filter results for the specific metric
    metric_data = results_df[results_df['Metric'] == metric_name]

    # Get data for pod_10 and pod_20
    pod_10 = metric_data[metric_data['Scenario'] == 'pod_10'].iloc[0] if len(metric_data[metric_data['Scenario'] == 'pod_10']) > 0 else None
    pod_20 = metric_data[metric_data['Scenario'] == 'pod_20'].iloc[0] if len(metric_data[metric_data['Scenario'] == 'pod_20']) > 0 else None

    latex = []
    latex.append("\\begin{table}[H]")
    latex.append("  \\centering")
    latex.append(f"  \\caption{{{caption}}}\\label{{{table_label}}}")
    latex.append("  \\begin{tabular}{lcc}")
    latex.append("    \\toprule")
    latex.append("    \\textbf{Statistik} & \\textbf{10 Pod} & \\textbf{20 Pod} \\\\")
    latex.append("    \\midrule")

    # N
    n_10 = int(pod_10['N']) if pod_10 is not None else "-"
    n_20 = int(pod_20['N']) if pod_20 is not None else "-"
    latex.append(f"    Jumlah sampel ($N$) & {n_10} & {n_20} \\\\")

    # HPA Mean
    if "Response Time" in metric_name:
        unit = " (ms)"
        hpa_10 = f"{pod_10['HPA Mean']:.3f}" if pod_10 is not None else "-"
        hpa_20 = f"{pod_20['HPA Mean']:.3f}" if pod_20 is not None else "-"
    elif "Replica" in metric_name:
        unit = ""
        hpa_10 = f"{pod_10['HPA Mean']:.3f}" if pod_10 is not None else "-"
        hpa_20 = f"{pod_20['HPA Mean']:.3f}" if pod_20 is not None else "-"
    else:  # CPU or Memory
        unit = " (\\%)"
        hpa_10 = f"{pod_10['HPA Mean']:.3f}" if pod_10 is not None else "-"
        hpa_20 = f"{pod_20['HPA Mean']:.3f}" if pod_20 is not None else "-"

    latex.append(f"    Rata-rata HPA{unit} & {hpa_10} & {hpa_20} \\\\")

    # RL Mean
    rl_10 = f"{pod_10['RL Mean']:.3f}" if pod_10 is not None else "-"
    rl_20 = f"{pod_20['RL Mean']:.3f}" if pod_20 is not None else "-"
    latex.append(f"    Rata-rata RL{unit} & {rl_10} & {rl_20} \\\\")

    # Diff (%)
    diff_10 = f"${pod_10['Diff (%)']:.3f}$" if pod_10 is not None else "-"
    diff_20 = f"${pod_20['Diff (%)']:.3f}$" if pod_20 is not None else "-"
    latex.append(f"    Selisih (\\%) & {diff_10} & {diff_20} \\\\")

    latex.append("    \\midrule")

    # Normality P-values
    norm_hpa_10 = f"{pod_10['Normality P(HPA)']:.3f}" if pod_10 is not None else "-"
    norm_hpa_20 = f"{pod_20['Normality P(HPA)']:.3f}" if pod_20 is not None else "-"
    latex.append(f"    \\emph{{P}}-value normalitas (HPA) & {norm_hpa_10} & {norm_hpa_20} \\\\")

    norm_rl_10 = f"{pod_10['Normality P(RL)']:.3f}" if pod_10 is not None else "-"
    norm_rl_20 = f"{pod_20['Normality P(RL)']:.3f}" if pod_20 is not None else "-"
    latex.append(f"    \\emph{{P}}-value normalitas (RL) & {norm_rl_10} & {norm_rl_20} \\\\")

    # Test used
    test_10 = f"\\emph{{{pod_10['Test']}}}" if pod_10 is not None else "-"
    test_20 = f"\\emph{{{pod_20['Test']}}}" if pod_20 is not None else "-"
    latex.append(f"    Uji yang digunakan & {test_10} & {test_20} \\\\")

    # P-value
    pval_10 = format_scientific(pod_10['P-Value']) if pod_10 is not None else "-"
    pval_20 = format_scientific(pod_20['P-Value']) if pod_20 is not None else "-"
    latex.append(f"    \\emph{{P}}-value & {pval_10} & {pval_20} \\\\")

    # Significant
    sig_10 = "Ya" if pod_10 is not None and pod_10['Significant'] else "Tidak"
    sig_20 = "Ya" if pod_20 is not None and pod_20['Significant'] else "Tidak"
    latex.append(f"    Signifikan ($\\alpha = 0{{,}}05$) & {sig_10} & {sig_20} \\\\")

    # Effect Size
    if pod_10 is not None and "Wilcoxon" in pod_10['Test']:
        effect_label = "$r$"
    else:
        effect_label = "Cohen's $d$"

    effect_10 = format_effect_size(pod_10['Effect Size'], pod_10['Test']) if pod_10 is not None else "-"
    effect_20 = format_effect_size(pod_20['Effect Size'], pod_20['Test']) if pod_20 is not None else "-"
    latex.append(f"    \\emph{{Effect size}} ({effect_label}) & {effect_10} & {effect_20} \\\\")

    latex.append("    \\bottomrule")
    latex.append("  \\end{tabular}")
    latex.append("\\end{table}")

    return "\n".join(latex)



In [ ]:

results = []
results = []
# ensure unpacking has no accidental stray commas: for folder, metric_dir, label, pod in METRICS:
for folder, metric_dir, label, pod in METRICS:
    normalized_folder = folder.replace("../", "").lstrip("/")
    path = os.path.join(DATA_DIR, normalized_folder, metric_dir)
    if not os.path.exists(path):
        print(f"Warning: data path not found, skipping: {path}")
        continue

    runs = get_run_means(path)
    hpa_vals, rl_vals = build_paired_from_runs(runs)

    if len(hpa_vals) >= 3 and len(rl_vals) >= 3:
        save_normality_plots(hpa_vals, rl_vals, f"{CHARTS_DIR}/pod_{pod}", metric_dir, label, pod)

    stats_res = analyze_stats(hpa_vals, rl_vals)

    if stats_res:
        # Normalize scenario to a short label so table generation can filter by 'pod_10' / 'pod_20'
        stats_res.update({"Scenario": f"pod_{pod}", "Metric": label})
        results.append(stats_res)
    print("")

if results:
    columns = ["Scenario", "Metric", "N", "HPA Mean", "RL Mean", "Diff (%)",
               "Normality P(HPA)", "Normality P(RL)",
               "P-Value", "Significant", "Test", "Effect Size"]
    print(pd.DataFrame(results)[columns].to_string())
else:
    print("No valid data found for analysis.")

# Prepare output tables only if we have results
os.makedirs("tables", exist_ok=True)
results_df = pd.DataFrame(results)

if results_df.empty:
    print("No results available — skipping LaTeX table generation.")
else:
    # Generate tables for each metric
    tables = [
        ("Response Time (ms)", "tab:inferensial-waktu-respon", "Hasil Uji Inferensial Waktu Respons", "inferensial_waktu_respon.tex"),
        ("Replica Count", "tab:inferensial-jumlah-replika", "Hasil Uji Inferensial Jumlah Replika", "inferensial_jumlah_replika.tex"),
        ("CPU Usage", "tab:inferensial-penggunaan-cpu", "Hasil Uji Inferensial Penggunaan CPU", "inferensial_penggunaan_cpu.tex"),
        ("Memory Usage", "tab:inferensial-penggunaan-memori", "Hasil Uji Inferensial Penggunaan Memori", "inferensial_penggunaan_memori.tex"),
    ]

    for metric_name, label, caption, filename in tables:
        # Safety: ensure required column exists
        if 'Metric' not in results_df.columns:
            print(f"Skipping table {filename}: 'Metric' column missing in results dataframe")
            continue

        latex_table = generate_latex_table(results_df, metric_name, label, caption)

        # Save to file
        with open(f"tables/{filename}", "w") as f:
            f.write(latex_table)

        print(f"Saved: tables/{filename}")
        print(latex_table)
        print("\n" + "="*80 + "\n")